In [1]:
import numpy as np
from scipy.linalg import solve_sylvester, norm
import scipy.sparse as sp

In [2]:
def newton_nare(A, B, C, D, X0, tol=1e-13, kmax=30):
    """
    Newton's method for solving the Nonlinear Algebraic Riccati Equation (NARE):
    C + XA + DX - XBX = 0
    """
    X = X0.copy()
    k = 0
    err = 1

    while err > tol and k < kmax:
        # Compute residual RX = C + XA + DX - XBX
        RX = C + X @ A + D @ X - X @ B @ X

        # Solve the Sylvester equation (D - XB)H + H(A - BX) = -RX for H
        H = solve_sylvester(D - X @ B, A - B @ X, -RX)

        # Update X
        X = X + H

        # Calculate the error; changed from l1 to frobenius
        err = norm(H, 1) / norm(X, 1)
        # err = norm(H, 'fro') / (1 + norm(X, 'fro'))

        if k % 5 == 0:  # Print every 5 iterations
            print(f"Iteration {k}, Error: {err:.2e}")
        
        # Increment iteration counter
        k += 1

    # Check if the solution converged
    if k == kmax:
        print("Warning: reached the maximum number of iterations without convergence.")

    return X

In [3]:
def ensure_positive_definite(M, epsilon=1e-3, min_threshold=1e-3):
    """Ensure M is well-conditioned ; add small value to diagonal if needed"""
    min_eig = np.min(np.linalg.eigvals(M))
    
    if min_eig < min_threshold:
        print(f"Minimum eigenvalue too small ({min_eig:.2e}), adding {epsilon} to diagonal elements.")
        M += np.eye(M.shape[0]) * (abs(min_eig) + epsilon)
    
    return M

In [11]:
# random dense matrices

n=100

A = np.zeros((n, n))

B = np.random.rand(n, n)
# symmetric positive definite
B = B.T @ B + np.eye(n) 
B = -B 
min_eig = np.min(np.linalg.eigvals(B))
print(f"Minimum eigenvalue ({min_eig:.2e})")

C = -2*np.eye(n)

# symmetric D
D = np.random.rand(n, n)
D = D.T @ D 

# initial guess
X0 = np.eye(n)

X_hat = newton_nare(A, B, C, D, X0)

residual = C + X_hat @ A + D @ X_hat - X_hat @ B @ X_hat
residual_norm = norm(residual, 'fro')

print('Residual after NARE: ', residual)
print('Its norm: ', residual_norm)

Minimum eigenvalue (-2.50e+03)
Iteration 0, Error: 1.02e+00
Iteration 5, Error: 1.08e-02
Residual after NARE:  [[-5.21804822e-15  2.33840725e-15 -2.45636844e-15 ...  2.13717932e-15
  -1.27675648e-15  2.84494650e-16]
 [ 9.33281230e-16  3.21964677e-15  2.10942375e-15 ...  2.13717932e-15
  -1.98452366e-15 -1.31838984e-15]
 [-2.83106871e-15  4.28823643e-15  8.43769499e-15 ...  4.99600361e-16
  -2.13717932e-15 -9.71445147e-16]
 ...
 [-8.18789481e-16  5.03069808e-15  9.71445147e-17 ...  6.43929354e-15
  -5.28743715e-15  1.92901251e-15]
 [ 1.49880108e-15  1.06026299e-14  1.58206781e-15 ...  1.80411242e-15
  -8.88178420e-16 -3.80251386e-15]
 [ 2.66453526e-15  5.05151476e-15  5.38458167e-15 ...  5.53723734e-15
   2.31759056e-15  1.11022302e-16]]
Its norm:  2.9363968579832017e-13


In [34]:
n=10

A = np.zeros((n, n))

B = np.array([
            [
                -59.11362531279102,
                75.43153594250221,
                -57.11124037763095,
                25.204210708824156,
                -11.580154625127848,
                7.09656811463124,
                -4.337148324936669,
                2.8326423337511,
                -1.5932758404398573,
                0.474695661540603
            ],
            [
                75.43153594250221,
                -100.43138209789133,
                75.45900932619696,
                -33.22103199709386,
                15.610176247077213,
                -9.545427137028039,
                5.84434913764021,
                -3.332290023245956,
                2.2315056301838685,
                -1.0337596756756418
            ],
            [
                -57.11124037763095,
                75.45900932619696,
                -59.534859422494016,
                26.22717395486668,
                -12.679620999632071,
                8.17552414800423,
                -4.902309918749398,
                2.6555675047751226,
                -2.064866595639905,
                0.7602448330458764
            ],
            [
                25.204210708824156,
                -33.22103199709386,
                26.22717395486668,
                -14.735953262918851,
                7.006843771595771,
                -4.726707846171816,
                2.4688014202526007,
                -1.1243268710458707,
                0.8008425547158349,
                -0.03780523648447405
            ],
            [
                -11.580154625127848,
                15.610176247077213,
                -12.679620999632071,
                7.006843771595771,
                -6.910613517806627,
                4.751807098556162,
                -2.670909224846079,
                1.0281224246207978,
                -0.7837517123092583,
                0.028146405802881677
            ],
            [
                7.09656811463124,
                -9.545427137028039,
                8.17552414800423,
                -4.726707846171816,
                4.751807098556162,
                -7.24148736693112,
                4.943800514960208,
                -2.8947965754444214,
                1.0183055863175776,
                0.083266331730018
            ],
            [
                -4.337148324936669,
                5.84434913764021,
                -4.902309918749398,
                2.4688014202526007,
                -2.670909224846079,
                4.943800514960208,
                -7.08180854957831,
                4.417206257314923,
                -1.2388811868644913,
                -0.001751057619573599
            ],
            [
                2.8326423337511,
                -3.332290023245956,
                2.6555675047751226,
                -1.1243268710458707,
                1.0281224246207978,
                -2.8947965754444214,
                4.417206257314923,
                -5.627169272636874,
                1.312608657174295,
                0.07512300153888427
            ],
            [
                -1.5932758404398573,
                2.2315056301838685,
                -2.064866595639905,
                0.8008425547158349,
                -0.7837517123092583,
                1.0183055863175776,
                -1.2388811868644913,
                1.312608657174295,
                -2.6901528579647493,
                0.256771385080583
            ],
            [
                0.474695661540603,
                -1.0337596756756418,
                0.7602448330458764,
                -0.03780523648447405,
                0.028146405802881677,
                0.083266331730018,
                -0.001751057619573599,
                0.07512300153888427,
                0.256771385080583,
                -2.2578746314865503
            ]
        ])
min_eig = np.min(np.linalg.eigvals(B))
print(f"Minimum eigenvalue of B ({min_eig:.2e})")

C = -2*np.eye(n)

# symmetric D
D = np.array([
            [
                -0.9026279364702661,
                -0.4526871310897436,
                -0.06346160943961326,
                0.1402907199683117,
                0.10898378291252539,
                -0.14073272086589925,
                0.11599866995826566,
                0.15542819849445944,
                0.14676619139234667,
                -0.15744821636846118
            ],
            [
                -0.44777625422043804,
                -0.820915833683252,
                -0.37017232213193624,
                -0.1815742217994533,
                0.15878341166220433,
                0.12275031669072153,
                0.07495845878948158,
                0.1186124256712161,
                -0.09591553062986882,
                -0.15973494959043888
            ],
            [
                -0.06901777130921888,
                -0.37488903335250906,
                -0.9103805089848097,
                -0.08583190108415673,
                -0.1485075806219906,
                0.15865432203866872,
                -0.1031760604852482,
                -0.0896976088805703,
                -0.1404409078641274,
                0.11950255830479842
            ],
            [
                0.12871034441923018,
                -0.1908406016354298,
                -0.08843555730053082,
                -0.8046025903943269,
                -0.042624649756746735,
                -0.17368022519237097,
                -0.096131835494128,
                0.11936352992553141,
                -0.1359575850703021,
                0.18333107304639737
            ],
            [
                0.048923815726967114,
                0.17430157591147707,
                -0.14167184577921302,
                -0.030465411844909307,
                -0.9164113959705685,
                -0.22470800962823362,
                0.07473593479287619,
                -0.12134709745709737,
                -0.1737809916876902,
                0.0008369954768345824
            ],
            [
                -0.09449790201771766,
                0.13811636635618715,
                0.1879463435225009,
                -0.14670910526795589,
                -0.2354610230502936,
                -0.9374909944220572,
                -0.2696154771779648,
                -0.17282442495140268,
                0.09258593554122502,
                0.09559219111452973
            ],
            [
                0.12343928726860379,
                0.11084747671227213,
                -0.10676529901648522,
                -0.13891439070423,
                0.04003799261671438,
                -0.25233643325869987,
                -0.9541916146656501,
                -0.2894120631289028,
                -0.12438846863551373,
                0.05718630626814524
            ],
            [
                0.15663768322561689,
                0.08673837062220395,
                -0.13427988000501587,
                0.12513516454137283,
                -0.13100372652851014,
                -0.10250468653086257,
                -0.2905456282842032,
                -0.8317072425333161,
                0.05003672446889834,
                0.10807475604280194
            ],
            [
                0.17692972390475742,
                -0.07282144395139624,
                -0.14948154397830707,
                -0.0736990371453808,
                -0.1890336833924164,
                0.15065075297862457,
                -0.17314112923839445,
                0.030385131369461277,
                -0.9477783483057095,
                0.15978485454784097
            ],
            [
                -0.18919880018425106,
                -0.17740020910464363,
                0.143572636459746,
                0.17505956149131116,
                -0.04609755822887462,
                0.09884586430886466,
                0.007514535686763914,
                0.09825351684848807,
                0.15913288114269566,
                -1.0036156611845453
            ]
        ])
min_eig = np.min(np.linalg.eigvals(D))
print(f"Minimum eigenvalue of D ({min_eig:.2e})")
print(f"Is D symmetric: {np.array_equal(D, D.T)}")

# initial guess
X0 = np.eye(n)

X_hat = newton_nare(A, B, C, D, X0)

residual = C + X_hat @ A + D @ X_hat - X_hat @ B @ X_hat
residual_norm = norm(residual, 'fro')

print('Residual after NARE: ', residual)
print('Its norm: ', residual_norm)

Minimum eigenvalue of B (-2.31e+02)
Minimum eigenvalue of D (-1.75e+00)
Is D symmetric: False
Iteration 0, Error: 7.07e-01
Iteration 5, Error: 3.98e-04
Residual after NARE:  [[-5.32907052e-15 -8.88178420e-15 -5.71764858e-15  8.60422844e-16
   5.68989300e-16  3.46944695e-16  2.22044605e-16  1.38777878e-16
   5.55111512e-17  3.60822483e-16]
 [ 2.44249065e-15  4.44089210e-16 -4.32986980e-15 -4.32986980e-15
  -7.77156117e-16 -6.93889390e-16 -1.38777878e-16 -2.77555756e-17
   9.43689571e-16 -5.55111512e-17]
 [-4.27435864e-15 -3.21964677e-15  2.22044605e-15  2.77555756e-15
   1.04083409e-15  1.31838984e-16  1.24900090e-16  0.00000000e+00
   4.30211422e-16  2.22044605e-16]
 [ 1.87003191e-15 -1.11022302e-15 -1.94289029e-15 -4.44089210e-16
   8.88178420e-16  3.88578059e-16  2.77555756e-17  4.85722573e-17
   1.11022302e-16  0.00000000e+00]
 [ 3.88578059e-16 -2.77555756e-16 -9.43689571e-16 -3.88578059e-16
  -8.88178420e-16 -1.11022302e-16  1.80411242e-16  6.93889390e-17
   8.32667268e-17 -1.73472